In [1]:
import pandas as pd


In [2]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import os
import time
import requests
import pandas as pd

df1 = pd.read_csv("Data/stock_analysis_screener_usa.csv")
df2 = pd.read_csv("Data/stock_analysis_screener_OTC_USA.csv")  

In [ ]:
url=df1['Link'][0]+"company/"
driver = webdriver.Chrome()
driver.get(url)
time.sleep(5)
x_path1 = '//*[@id="main"]/div[2]/div[1]'
# Extract the text
description_element = driver.find_element(By.XPATH, x_path1)
description_text = description_element.text


x_path2 = '//*[@id="main"]/div[2]/div[2]'
# Extract the element containing the company details
company_details_element = driver.find_element(By.XPATH, x_path2)

# Extract company name
company_name = company_details_element.find_element(By.CLASS_NAME, "text-2xl").text

# Extract the logo image URL and save it
logo_element = company_details_element.find_element(By.TAG_NAME, "img")
logo_url = logo_element.get_attribute("src")

# Directory to save the logo
logo_dir = r"Data\company_logos_stock_analysis"
os.makedirs(logo_dir, exist_ok=True)

# Save the logo image
logo_filename = os.path.join(logo_dir, f"{company_name.replace(' ', '_')}_logo.svg")
response = requests.get(logo_url)
if response.status_code == 200:
    with open(logo_filename, "wb") as logo_file:
        logo_file.write(response.content)
    print(f"Logo saved at: {logo_filename}")
else:
    print(f"Failed to download logo from {logo_url}")



# Extract rows from the table
rows = company_details_element.find_elements(By.TAG_NAME, "tr")

# Parse table rows into a dictionary
company_details = {"Company Name": company_name}

company_details['Company description'] = description_text.split("\n", 1 )[-1]

for row in rows:
    cells = row.find_elements(By.TAG_NAME, "td")
    if len(cells) == 1:
        if  "Address:" in row.text.strip():
            key = "Address"
            value = row.text.replace( "Address:","").strip()
            company_details[key] = value
        
    if len(cells) == 2:
        key = cells[0].text.strip()
        value = cells[1].text.strip()
        company_details[key] = value

driver.quit()


Logo saved at: Data\company_logos_stock_analysis\Vertex_Pharmaceuticals_Incorporated_logo.svg


In [26]:

for key, value in company_details.items():
    print("*"*50)
    print(f"{key}: {value}")

**************************************************
Company Name: Vertex Pharmaceuticals Incorporated
**************************************************
Company description: Vertex Pharmaceuticals Incorporated, a biotechnology company, engages in developing and commercializing therapies for treating cystic fibrosis (CF).
It markets TRIKAFTA/KAFTRIO for people with CF with at least one F508del mutation for 2 years of age or older; SYMDEKO/SYMKEVI for people with CF for 6 years of age or older; ORKAMBI for CF patients 1 year or older; and KALYDECO for the treatment of patients with 1 year or older who have CF with ivacaftor.
The company’s pipeline includes VX-522, a CFTR mRNA therapeutic designed to treat the underlying cause of CF, which is in Phase 1 clinical trial; VX-548, a non-opioid medicine for the treatment of acute and neuropathic pain which is in Phase 3 clinical trial; Exa-cel, for the treatment of sickle cell disease and transfusion-dependent beta thalassemia which is in Phase

In [3]:
df1 = pd.read_csv("Data/stock_analysis_screener_usa.csv")
df2 = pd.read_csv("Data/stock_analysis_screener_OTC_USA.csv")

# Combine dataframes
all_companies = pd.concat([df1, df2], ignore_index=True)
